In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
2+2

4

In [3]:
from tqdm.auto import tqdm
import pandas as pd

In [4]:
import json
import sys
import os
from dotenv import load_dotenv
# Load environment variables from .env
load_dotenv("/home/jovyan/.envrc")

True

In [5]:
# Construct the path to the scripts directory
script_path = os.path.abspath('../reviews-assistant/scripts')

# Add the path to sys.path
if script_path not in sys.path:
    sys.path.append(script_path)
    
import minsearch

In [6]:
# Directory containing the ground truth file
data_dir = os.path.abspath('../reviews-assistant/data/ground_truth')

# Path to the ground_truth_retrieval.json file
file_path = os.path.join(data_dir, 'ground_truth_retrieval.json')

# Check if the file exists
if os.path.exists(file_path):
    try:
        # Open and load the JSON file
        with open(file_path, 'r', encoding='utf-8') as jsonfile:
            ground_truth_data = json.load(jsonfile)  # Load the JSON data into a Python object

        # Ensure each item is a dictionary and contains 'id' and 'question'
        if all(isinstance(item, dict) and 'appid' in item and 'question' in item for item in ground_truth_data):
            print("Data successfully loaded and is in the correct format.")

            # Example: Print a sample of the data
            for item in ground_truth_data[:5]:  # Print the first 5 questions
                print(f"ID: {item['appid']}, Question: {item['question']}")
        else:
            print("Error: The data format is incorrect or missing required fields ('id', 'question').")
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
else:
    print(f"File {file_path} does not exist.")

Data successfully loaded and is in the correct format.
ID: 2239550, Question: What are the content themes present in Watch Dogs: Legion that I should be aware of?
ID: 2239550, Question: Does Watch Dogs: Legion contain any excessive violence or graphic depictions?
ID: 2239550, Question: Are there any instances of strong language or profanity in Watch Dogs: Legion?
ID: 2239550, Question: Does Watch Dogs: Legion include any substance abuse or gambling mechanics?
ID: 2239550, Question: Is there any disturbing imagery or horror elements in Watch Dogs: Legion?


In [7]:
len(ground_truth_data)

105

In [8]:
# Directory containing the data files
data_dir = os.path.abspath('../reviews-assistant/data/reviews')

# Initialize an empty list to hold all reviews
reviews = []

# List objects in the directory
objects_in_directory = os.listdir(data_dir)

# Iterate over the files in the directory
for obj in objects_in_directory:
    if obj.endswith('.json'):  # Check if the file is a JSON file
        file_path = os.path.join(data_dir, obj)
        with open(file_path, 'r', encoding='utf-8') as jsonfile:
            # Load the reviews from the JSON file
            file_reviews = json.load(jsonfile)
            reviews.extend(file_reviews)  # Append reviews to the main list
# Print the first i reviews
i = 2  # Change this to print more reviews if needed
for review in reviews[:i]:
    print(f"Author ID: {review['author.steamid']}")
    print(f"Review: {review.get('review', 'No text')}")
    print(f"Timestamp Created: {review['timestamp_created']}")
    print("-" * 79)

Author ID: 76561198420943538
Review: ---{ Graphics }---
✅ You forget what reality is
☐ Beautiful
☐ Good
☐ Decent
☐ Bad
☐ You will get eye cancer
☐ Get a pepper spray for your eye instead

---{ Gameplay }---
☐ Won’t ever touch any other game anymore
✅ Very good
☐ Good
☐ It's just gameplay
☐ Mehh
☐ Watch paint dry instead
☐ Tic Tac toe is better

---{ Audio }---
☐ Eargasm
✅ Very good
☐ Good
☐ Not too bad
☐ Bad
☐ I'm now deaf

---{ Audience }---
☐ Kids
✅Teens
✅ Adults
☐ Grandma

---{ PC Requirements }---
☐ Check if you can run paint
☐ Potato
☐ Decent
✅ Fast
☐ Rich boi
☐ Ask NASA if they have a spare computer
☐ Search the galaxy for dark matter fuel to run

---{ Difficulty }---
☐ Just press 'W'
☐ Easy
✅ Easy to learn / Hard to master
☐ Significant brain usage
☐ Difficult
☐ Dark Souls

---{ Grind }---
☐ Nothing to grind
☐ Only if u care about leaderboards/ranks
✅ Isn't necessary to progress
☐ Average grind level
☐ Too much grind
☐ You'll need a second life for grinding

---{ Story }---
☐ No

In [9]:
len(reviews)

1782

In [10]:
index = minsearch.Index(
    # text_fields=["author.steamid", "author.playtimeforever", "author.playtime_last_two_weeks", "author.playtime_at_review", "author.last_played", "language", "review", "timestamp_created", "timestamp_updated"],
    text_fields=["title", "language", "review"],
    keyword_fields=["appid", "recommendationid"]
)

index.fit(reviews)

In [11]:
from openai import OpenAI

client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  # this is also the default, it can be omitted
)

In [12]:
def build_prompt(query, search_results):
    answer_llm = ""

    if 'entry_template' in globals():  # Check if entry_template exists in global scope
        for doc in search_results:
            answer_llm += entry_template.format(**doc) + "\n\n"
    else:
        for doc in search_results:
            # Fallback formatting if entry_template is missing
            answer_llm += str(doc) + "\n\n"

    prompt = prompt_template.format(question=query, answer_llm=answer_llm).strip()
    return prompt

In [13]:
def llm(prompt, model='gpt-4o-mini'):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [14]:
def search(query):
    boost = {'exercise_name': 0.6625233926839482,
  'type_of_activity': 1.696177528847092,
  'type_of_equipment': 2.5131837819420255,
  'body_part': 1.7285666348004312,
  'type': 2.7764448064716354,
  'muscle_groups_activated': 2.1997970868248142,
  'instructions': 1.7565742031203238}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [15]:
def rag(query, model='gpt-4o-mini'):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    #print(prompt)
    answer = llm(prompt, model=model)
    return answer

In [16]:
prompt_template = """
You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{{
  "relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()

In [17]:
# sample = ground_truth_data

In [18]:
# record['question']

In [19]:
# rag(question)

In [20]:
# answer_llm = rag(question)

In [21]:
evaluations = []

for record in tqdm(ground_truth_data):
    question = record['question']
    answer_llm = rag(question) 

    prompt = prompt_template.format(
        question=question,
        answer_llm=answer_llm
    )

    evaluation = llm(prompt)
    evaluation = json.loads(evaluation)

    evaluations.append((record, answer_llm, evaluation))

  0%|          | 0/105 [00:00<?, ?it/s]

In [22]:
evaluations[0][0]['question']

'What are the content themes present in Watch Dogs: Legion that I should be aware of?'

In [23]:
evaluations[:1]

[({'appid': '2239550',
   'question': 'What are the content themes present in Watch Dogs: Legion that I should be aware of?'},
  '{\n  "relevance": "RELEVANT",\n  "explanation": "The generated answer provides a comprehensive overview of the content themes in \'Watch Dogs: Legion\', including its innovative mechanics, dystopian setting, and the various factions that represent different societal issues. It touches on the themes of surveillance, resistance against authoritarianism, and the ethical implications of technology, which are all central to the game\'s narrative. This directly addresses the question regarding the content themes one should be aware of."\n}',
  {'relevance': 'RELEVANT',
   'explanation': "The generated answer thoroughly addresses the question by outlining key content themes present in 'Watch Dogs: Legion'. It highlights important aspects such as the dystopian setting, various factions, and critical themes like surveillance, resistance, and ethical considerations re

In [24]:
df_eval = pd.DataFrame(evaluations, columns=['record', 'answer', 'evaluation'])

df_eval['appid'] = df_eval.record.apply(lambda d: d['appid'])
df_eval['question'] = df_eval.record.apply(lambda d: d['question'])

df_eval['relevance'] = df_eval.evaluation.apply(lambda d: d['relevance'])
df_eval['explanation'] = df_eval.evaluation.apply(lambda d: d['explanation'])

del df_eval['record']
del df_eval['evaluation']

In [25]:
df_eval.relevance.value_counts()

relevance
NON_RELEVANT       80
RELEVANT           18
PARTLY_RELEVANT     7
Name: count, dtype: int64

In [26]:
df_eval.relevance.value_counts(normalize=True)

relevance
NON_RELEVANT       0.761905
RELEVANT           0.171429
PARTLY_RELEVANT    0.066667
Name: proportion, dtype: float64

In [27]:
import json
import os
import pandas as pd

# Sample DataFrame for demonstration
# df_eval = pd.DataFrame({'recommendationid': [1, 2], 'question': ['What is the best game?', 'What is the worst game?']})

data_dir = os.path.abspath('../reviews-assistant/data/ground_truth')

# Define the output file path
output_file = os.path.join(data_dir, "ground_truth_evaluation.json")

# Convert DataFrame to a list of dictionaries including index
df_eval_list = df_eval.reset_index().to_dict(orient='records')

# Save the results to a JSON file
with open(output_file, 'w', encoding='utf-8') as json_file:
    json.dump(df_eval_list, json_file, ensure_ascii=False, indent=4)

print(f"Data saved to {output_file}")

Data saved to /home/jovyan/reviews-assistant/data/ground_truth/ground_truth_evaluation.json


In [28]:
df_eval[df_eval["relevance"]=="RELEVANT"]

,answer,appid,question,relevance,explanation
0,"{\n ""relevance"": ""RELEVANT"",\n ""explanation""...",2239550,What are the content themes present in Watch D...,RELEVANT,The generated answer thoroughly addresses the ...
2,"{\n ""relevance"": ""RELEVANT"",\n ""explanation""...",2239550,Are there any instances of strong language or ...,RELEVANT,The generated answer directly addresses the qu...
20,"{\n ""relevance"": ""RELEVANT"",\n ""explanation""...",1680880,Is Forspoken suitable for my children given th...,RELEVANT,The generated answer directly addresses the qu...
25,"{\n ""relevance"": ""RELEVANT"",\n ""explanation""...",2369390,What concerns does this game raise regarding t...,RELEVANT,The generated answer directly pertains to the ...
27,"{\n ""relevance"": ""RELEVANT"",\n ""explanation""...",2369390,How does the gameplay design in Far Cry 6 affe...,RELEVANT,The generated answer effectively addresses the...
28,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",2369390,Does the review mention any potentially harmfu...,RELEVANT,The generated answer accurately addresses the ...
34,"{\n ""relevance"": ""RELEVANT"",\n ""explanation""...",315210,Are there any features in this game that encou...,RELEVANT,The generated answer directly addresses the qu...
48,"{\n ""relevance"": ""RELEVANT"",\n ""explanation""...",1832040,Does the game include strong language or profa...,RELEVANT,The generated answer directly addresses the qu...
49,"{\n ""relevance"": ""RELEVANT"",\n ""explanation""...",1832040,Are there any elements in this game that promo...,RELEVANT,The generated answer effectively addresses the...
54,"{\n ""relevance"": ""RELEVANT"",\n ""explanation""...",304390,Is there any use of strong language or profani...,RELEVANT,The generated answer directly addresses the qu...


In [29]:
df_eval[df_eval["relevance"]=="NON_RELEVANT"]

,answer,appid,question,relevance,explanation
1,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",2239550,Does Watch Dogs: Legion contain any excessive ...,NON_RELEVANT,The generated answer fails to address the spec...
3,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",2239550,Does Watch Dogs: Legion include any substance ...,NON_RELEVANT,The generated answer fails to address the spec...
4,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",2239550,Is there any disturbing imagery or horror elem...,NON_RELEVANT,The generated answer does not address the spec...
5,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",243470,What kind of content does Watch_Dogs include t...,NON_RELEVANT,The generated answer does not address the spec...
6,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",243470,Are there any instances of violence or graphic...,NON_RELEVANT,The response does not provide any relevant inf...
...,...,...,...,...,...
100,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",794540,Does Neo Cab contain excessive or graphic depi...,NON_RELEVANT,The generated answer does not address the spec...
101,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",794540,Are there any elements of gore or blood in Neo...,NON_RELEVANT,The generated answer does not address the spec...
102,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",794540,Does the game include any explicit or suggesti...,NON_RELEVANT,The generated answer completely fails to addre...
103,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",794540,What kind of language is used in Neo Cab? Is t...,NON_RELEVANT,The generated answer fails to address the spec...


In [30]:
df_eval[df_eval["appid"]=="2322010"]

,answer,appid,question,relevance,explanation
40,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",2322010,What type of harmful content is present in God...,NON_RELEVANT,The generated answer fails to address the spec...
41,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",2322010,Is God of War: Ragnarok suitable for a young a...,NON_RELEVANT,The generated answer does not pertain to the q...
42,"{\n ""relevance"": ""PARTLY_RELEVANT"",\n ""expla...",2322010,Does the game contain any gambling elements or...,PARTLY_RELEVANT,The generated answer touches on aspects relate...
43,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",2322010,Are there any explicit themes or strong langua...,NON_RELEVANT,The generated answer does not address the ques...
44,"{\n ""relevance"": ""RELEVANT"",\n ""explanation""...",2322010,Can I play God of War: Ragnarok on Steam witho...,PARTLY_RELEVANT,The generated answer discusses the need for a ...


In [31]:
df_eval.iloc[8:9]

,answer,appid,question,relevance,explanation
8,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",243470,Is there any sexual content or suggestive them...,NON_RELEVANT,The generated answer fails to address the spec...


In [32]:
df_eval.iloc[8:9]["question"]

8    Is there any sexual content or suggestive them...
Name: question, dtype: object